In [1]:
from importlib import reload
import os 
import pathlib
import sys
sys.path.append('..')

from kerchunk.hdf import SingleHdf5ToZarr 
from kerchunk.combine import MultiZarrToZarr

import cartopy.crs as ccrs
import cartopy.feature as cfeature 
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import dask
from dask.distributed import Client
from dask.diagnostics import ProgressBar

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import s3fs
import datetime as dt

import logging
import fsspec
import ujson
from tqdm import tqdm
from glob import glob
import seaborn as sb
from IPython.display import display, clear_output

from src import processing as pr, parallel_plotting as pp
reload(pr)
reload(pp)

<module 'src.parallel_plotting' from '/Users/geet/Documents/Repositories/Owned/goesdata/notebooks/../src/parallel_plotting.py'>

In [2]:
fs = fsspec.filesystem('s3', anon=True)
urls = []

for i in tqdm(range(180,210)):
    for f in fs.glob(f"s3://noaa-goes16/ABI-L2-CMIPF/2022/{i}/*/*C13*.nc"):
        urls.append('s3://' + f)

100%|██████████| 30/30 [01:57<00:00,  3.91s/it]


In [4]:
client = Client(n_workers=8)
client

/Users/geet/Documents/Repositories/Owned/goesdata/goesdata_env/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55486 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:55486/status,
Dashboard: http://127.0.0.1:55486/status,Workers: 8
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55488,Workers: 8
Dashboard: http://127.0.0.1:55486/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:55510,Total threads: 1
Dashboard: http://127.0.0.1:55511/status,Memory: 2.00 GiB
Nanny: tcp://127.0.0.1:55491,


In [5]:
def gen_json(u):
    so = dict(
        mode="rb", anon=True, default_fill_cache=False,
        default_cache_type="none"
    )
    with fsspec.open(u, **so) as inf:
        h5chunks = SingleHdf5ToZarr(inf, u, inline_threshold=300)
        if os.path.isfile(f"jsons/{u.split('/')[-1]}.json"):
            pass
        else:
            with open(f"jsons/{u.split('/')[-1]}.json", 'wb') as outf:
                outf.write(ujson.dumps(h5chunks.translate()).encode())
            # clear_output(wait=True)
            # display(f"Last created: jsons/{u.split('/')[-1]}.json")

In [6]:
pathlib.Path('./jsons/').mkdir(exist_ok=True)

with ProgressBar():
    dask.compute(*[dask.delayed(gen_json)(u) for u in urls])

KeyboardInterrupt: 